In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_ppm(ppm_path: str) -> np.ndarray:
    with open(ppm_path, 'rb') as ppm:
        header = ppm.readline().decode()
        if header.strip() != 'P6':
            raise Exception('Only Raw PPM file supported.')
        
        metadata_count = 0
        metadata = []
        while metadata_count < 2:
            line = ppm.readline().decode()
            if line.startswith('#'):
                continue
            
            metadata.extend([int(v) for v in line.split(' ')])
            metadata_count += 1
            
        width, height, maxval = metadata

        if maxval > 255:
            raise Exception('Only 8-bit images supported.')


        return np.frombuffer(ppm.read(), dtype=np.uint8).reshape((height, width, 3))

def apply_filter(img_channel: np.ndarray, filter_size: int, filt: np.ndarray) -> np.ndarray:
    filt = np.flip(filt, axis=(0,1))
    pad_size = filter_size // 2
    padded_img = np.pad(img_channel.astype(np.float32), pad_size, mode='constant')
    height, width = padded_img.shape

    for i in range(pad_size, height - pad_size):
        for j in range(pad_size, width - pad_size):
            chunk = padded_img[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
            padded_img[i, j] = np.sum(chunk * filt)

    output = padded_img[pad_size:height - pad_size, pad_size:width - pad_size]
    return np.clip(output, 0, 255).astype(np.uint8)


def filter_ppm(img: np.ndarray, filter_size: int, filt: np.ndarray) -> np.ndarray:
    filtered_channels = []
    for c in range(3):
        filtered_channels.append(apply_filter(img[:, :, c], filter_size, filt))
    
    return np.stack(filtered_channels, axis=-1)

def get_identity_kernel(size: int) -> np.ndarray:
    if size % 2 == 0:
        raise Exception("Kernel must have an odd size.")
    if size <= 1:
        raise Exception("Kernels must have size 3 or greated (and should be odd).")

    identity = np.zeros((size, size), dtype=np.float32)
    identity[size//2, size//2] = 1
    return identity

# Referência: 
# https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy
def get_gaussian_kernel(size=5, sigma=1):
    """
    Creates gaussian kernel with side length `l` and a sigma of `sig`
    """
    ax = np.linspace(-(size - 1) / 2.0, (size - 1) / 2.0, size)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

In [ ]:
img_path = '../ppm_images/west_1.ppm'
img = read_ppm(img_path)
# img_path = '../images/portrair.jpg'
# img = np.array(Image.open(img_path))
img.shape

n = 5
mean_kernel = 1 / 9 * np.ones((n, n), dtype=np.float32)
identity = get_identity_kernel(n) 
gauss_kernel = get_gaussian_kernel(n)

sharp_basic = 2 * identity - mean_kernel
sharp_gauss = 2 * identity - gauss_kernel

# filtered = filter_ppm(img, n, sharp_gauss)

# # Show filtered
# fi = Image.fromarray(filtered)
# fi.show()

# Show Original
with Image.open(img_path, 'r') as img:
    img.show()

In [ ]:
# Test simple filter
# fake_img = np.array(
#     [[234,186,215,188,101,252,24,237,215],
#     [65,75,231,220,168,87,164,210,48],
#     [163,177,40,206,2,161,163,65,32],
#     [138,46,191,194,181,233,147,65,151],
#     [174,178,198,75,142,235,210,4,93],
#     [240,180,239,192,24,201,171,191,214],
#     [106,214,142,104,246,246,221,159,167],
#     [58,132,116,216,145,97,228,78,63],
#     [166,254,216,201,150,182,57,153,80]],
# )
# n = 3
# mean_filter = 1 / 9 * np.ones((n, n), dtype=np.float32)
# apply_filter(fake_img, n, mean_filter)